In [1]:
import pandas as pd
import numpy as np
import ast
import warnings

pd.set_option("display.max_columns", None) # MUESTRA TODAS LAS COLUNAS DE UN PANDAS DATAFRAME
warnings.filterwarnings('ignore')

# Procedemos con la ingesta del archivo "movies_dataset.csv"

In [2]:
data = pd.read_csv(r'./Dataset/movies_dataset.csv')

In [3]:
# Cambiando nombre de columnas a minusculas
data = data.rename(str.lower, axis='columns')

In [4]:
# Chequeando data duplicada
print('Registros duplicados:', data.duplicated().sum())

# Borrando registros duplicados
data.drop_duplicates(inplace=True)

Registros duplicados: 13


In [5]:
# Revisando registros con errores en campo "adult"
data[ (data['adult'] != 'True') & (data['adult'] != 'False') ]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Revisando registros con errores en campo "adult", se ha detectado datos incorrectos en la data, se procede a eliminarlo
filtroBorrar = data[ (data['adult'] != 'True') & (data['adult'] != 'False') ].index
data.drop(filtroBorrar, inplace = True)

In [7]:
# Creando una funcion generica que permite convertir un lista de diccionarios en un string separados por comas
# antes deben convertir los datos nulos como "sin dato"

def convertListaDict_String(x, name_campo):
    if x == 'sin dato':
        return x
    else:
        salida = []
        mi_lista = ast.literal_eval(x)
        for item in mi_lista:
            mi_dict = ast.literal_eval(str(item))
            salida.append(mi_dict.get(name_campo))
        return ', '.join(salida)

def convertDict_String(x, name_campo):
    if x == 'sin dato':
        salida = x
    else:
        mi_dict = ast.literal_eval(x)
        salida = mi_dict.get(name_campo)
        
    return salida

Revisando valores nulos y convirtiendo la lista de diccionarios a string
- belongs_to_collection : Un diccionario que indica a que franquicia o serie de películas pertenece la película
- budget : El presupuesto de la película, en dólares
- genres : Un diccionario que indica todos los géneros asociados a la película
- production_companies : Lista con las compañias productoras asociadas a la película
- production_countries : Lista con los países donde se produjo la película
- spoken_languages : Lista con los idiomas que se hablan en la pelicula

In [8]:
columnas = ['belongs_to_collection', 'budget', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
for col in columnas:
    # Buscando cantidad de valores nulos
    print('Valores nulos en',col, data[col].isnull().sum())
    # Reemplazando valor nulo con "sin dato"
    data[col].fillna('sin dato', inplace = True)

Valores nulos en belongs_to_collection 40959
Valores nulos en budget 0
Valores nulos en genres 0
Valores nulos en production_companies 3
Valores nulos en production_countries 3
Valores nulos en spoken_languages 3


In [9]:
# Convirtiendo la lista de diccionarios en string
for col in columnas:
    if col == 'belongs_to_collection':
        data['_' + col] = data[col].apply(lambda x: convertDict_String(x, 'name'))    
    elif col in ['genres', 'production_companies']:
        data['_' + col] = data[col].apply(lambda x: convertListaDict_String(x, 'name'))
    elif col in ['production_countries']:
        data['_' + col] = data[col].apply(lambda x: convertListaDict_String(x, 'name'))
    elif col in ['spoken_languages']:
        data['_' + col] = data[col].apply(lambda x: convertListaDict_String(x, 'name')) 

In [10]:
# Convirtiendo el tipo de dato de las columnas
data['id'] = data['id'].astype('int')
data['budget'] = data['budget'].astype('float')
data['revenue'] = data['revenue'].astype('float')
data['popularity'] = data['popularity'].astype('float')
data['_production_countries'] = data['_production_countries'].str.lower()

Propuesta de trabajo (requerimientos de aprobación) Transformaciones

In [11]:
# Algunos campos, como belongs_to_collection, production_companiesy otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desinfectarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desinfectarlos.
print('Se termino de haces las transformaciones requeridas en la seccion anterior')

Se termino de haces las transformaciones requeridas en la seccion anterior


In [12]:
# Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
print(data['revenue'].isna().sum())
print(data['budget'].isna().sum())

data.drop(data[data['revenue'].isna()].index, inplace=True)
data.drop(data[data['budget'].isna()].index, inplace=True)

3
0


In [13]:
# Los valores nulos del campo release_date deben eliminarse.
print(data['release_date'].isna().sum())
data.drop(data[data['release_date'].isna()].index, inplace=True)

84


In [14]:
# De haber fechas, deberá tener el formato AAAA-mm-dd, además deberá crear la columna release_year donde extraerán el año de la fecha de estreno.
data['release_date'] = pd.to_datetime(data['release_date'])
data['release_year'] = data['release_date'].dt.year

In [15]:
# Cree la columna con el retorno de inversión, llame return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no haya datos disponibles para calcularlo, deberá tomar el valor 0.
def retornoInversion(revenue, budget):
    if budget == 0:
        return 0
    else:
        return revenue / budget

data['return'] = data.apply(lambda row : retornoInversion(row['revenue'], row['budget']), axis = 1)

In [16]:
# Eliminar las columnas que no se utilizarán, video, imdb_id, adult, original_title, poster_path y homepage.
data.drop(['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)

In [17]:
# Eliminando columnas que no son necesarios, porque tiene su equivalente calculado
data.drop(['genres', 'belongs_to_collection', 'production_companies', 'production_countries','spoken_languages'], axis=1, inplace=True)

In [18]:
categorica_columns = data.select_dtypes(include = ["object","category"])
categorica_columns.isna().sum()

original_language            11
overview                    941
status                       80
tagline                   24969
title                         0
_belongs_to_collection        0
_genres                       0
_production_companies         0
_production_countries         0
_spoken_languages             0
dtype: int64

In [19]:
# Reemplazando valor nulo con "sin dato" en campos categoricos
columnas = categorica_columns.columns.values
for col in columnas:
    data[col].fillna('sin dato', inplace = True)

# Procedemos con la ingesta del archivo "credits.csv"

In [20]:
dataCredits = pd.read_csv(r'./Dataset/credits.csv')

In [21]:
dataCredits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [22]:
# Chequeando data duplicada
print('Registros duplicados:', dataCredits.duplicated().sum())

# Borrando registros duplicados
dataCredits.drop_duplicates(inplace=True)

Registros duplicados: 37


Revisando valores nulos y convirtiendo la lista de diccionarios a string
- cast : actores de la pelicula
- crew : directores de la pelicula

In [23]:
columnas = ['cast', 'crew']
for col in columnas:
    # Buscando cantidad de valores nulos
    print('Valores nulos en',col, dataCredits[col].isnull().sum())
    # Reemplazando valor nulo con "sin dato"
    dataCredits[col].fillna('sin dato', inplace = True)

Valores nulos en cast 0
Valores nulos en crew 0


In [24]:
# Creando una funcion que permite convertir un lista de diccionarios en un string separados por comas, filtrando por "job"
# antes deben convertir los datos nulos como "sin dato"
def convertListaDictCrew_String(x, name_campo, job):
    if x == 'sin dato':
        return x
    else:
        salida = []
        mi_lista = ast.literal_eval(x)
        for item in mi_lista:
            mi_dict = ast.literal_eval(str(item))
            if mi_dict.get('job') == job:
                salida.append(mi_dict.get(name_campo))
        return ', '.join(salida)

In [25]:
# Convirtiendo la lista de diccionarios en string
for col in columnas:
    if col == 'cast':
        dataCredits['actor'] = dataCredits['cast'].apply(lambda x: convertListaDict_String(x, 'name'))
    elif col == 'crew':
        dataCredits['director'] = dataCredits['crew'].apply(lambda x: convertListaDictCrew_String(x, 'name', 'Director'))

In [26]:
dataCredits.sample(2)

,cast,crew,id,actor,director
22180,"[{'cast_id': 16, 'character': 'Luis', 'credit_...","[{'credit_id': '5681d3fc9251414f6300b345', 'de...",54236,"José Luis López Vázquez, Lina Canalejas, Ferna...",Carlos Saura
29878,"[{'cast_id': 18, 'character': 'Herself', 'cred...","[{'credit_id': '57c8b06fc3a3685bec0009cf', 'de...",333382,"Violet Blue, Alan M. Dershowitz, David Greenfi...",Erin Lee Carr


In [27]:
dataCredits_unicos = dataCredits[['id', 'actor', 'director']].copy()

In [28]:
# Chequeando data duplicada
print('Registros duplicados:', dataCredits_unicos.duplicated().sum())

# Borrando registros duplicados
dataCredits_unicos.drop_duplicates(inplace=True)

Registros duplicados: 6


In [29]:
dataCredits_unicos['id'] = dataCredits_unicos['id'].astype('int')

In [30]:
data_final = data.merge(dataCredits_unicos[['id', 'actor', 'director']], on='id', how='left')

In [31]:
data_final.sample(2)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,_belongs_to_collection,_genres,_production_companies,_production_countries,_spoken_languages,release_year,return,actor,director
34918,0.0,336884,no,An old sanatorium is deteriorating in an isola...,1.395306,2015-10-09,0.0,90.0,Released,"They can demolish a building, but never remove...",Dark Woods II,4.7,14.0,Dark Woods Collection,Horror,Filmkompaniet MadMonkey,norway,Norsk,2015,0.0,"Anders Baasmo Christiansen, Ellen Dorrit Peter...",Pål Øie
39545,0.0,253343,en,Two teenage girls travel across the U.S. in 19...,2.626192,2016-07-05,0.0,89.0,Post Production,sin dato,Dear Eleanor,6.2,25.0,sin dato,"Adventure, Comedy, Drama",,,English,2016,0.0,"Liana Liberato, Isabelle Fuhrman, Josh Lucas, ...",Kevin Connolly


In [30]:
data_final.to_csv('./Dataset/movies_final.csv')

# API funciones

In [32]:
df = pd.read_csv(r'./Dataset/movies_final.zip', parse_dates=['release_date'])

In [33]:
df.sample(2)

,Unnamed: 0,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,_belongs_to_collection,_genres,_production_companies,_production_countries,_spoken_languages,release_year,return,actor,director
6392,6392,0.0,32235,en,"""We are all angels. It is what we do with our ...",3.450973,2003-01-01,0.0,103.0,Released,NaN,Northfork,6.8,26.0,sin dato,"Fantasy, Drama, Science Fiction",NaN,united states of america,English,2003,0.0,"James Woods, Nick Nolte, Claire Forlani, Daryl...",Michael Polish
9057,9057,0.0,152426,en,"On a red eye flight from LA to Boston, 10 peop...",12.215925,1995-05-14,0.0,180.0,Released,Prepare yourself for the flight of your life!,The Langoliers,6.0,158.0,sin dato,"Drama, Horror, Science Fiction, Mystery, Thriller","Laurel Entertainment, Spelling Films, Worldvision",united states of america,"English, Magyar, Español",1995,0.0,"Patricia Wettig, Dean Stockwell, David Morse, ...",Tom Holland


In [34]:
def cantidad_filmaciones_mes(Mes: str): 
    # Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron
    # estrenadas en el mes consultado en la totalidad del conjunto de datos.
    # Ejemplo de retorno: X cantidad de peliculas fueron estrenadas en el mes de X
    mes_lista = {'enero': 1, 
                'febrero': 2,
                'marzo': 3,
                'abril': 4,
                'mayo': 5,
                'junio': 6,
                'julio': 7,
                'agosto': 8,
                'setiembre': 9,
                'octubre': 10,
                'noviembre': 11,
                'diciembre': 12}
    # Validando el mes
    Mes = Mes.lower()
    if not Mes in mes_lista:
        return 'Se debe ingresar un mes en idioma español.'
    mes_nro = mes_lista.get(Mes)
    cantidad = len(df[df['release_date'].dt.month == mes_nro])
    
    return {'mes':Mes, 'cantidad':cantidad}

In [35]:
cantidad_filmaciones_mes('febrero')

{'mes': 'febrero', 'cantidad': 3029}

In [36]:
def cantidad_filmaciones_dia(Dia: str):
    # Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que
    # fueron estrenadas en el día consultado en la totalidad del conjunto de datos.
    # Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X
    dia_lista = {'lunes': 0, 
            'martes': 1,
            'miercoles': 2,
            'jueves': 3,
            'viernes': 4,
            'sabado': 5,
            'domingo': 6}
    
    # Validando el dia
    Dia = Dia.lower()
    if not Dia in dia_lista:
        return 'Se debe ingresar un dia en idioma español.'
    dia_nro = dia_lista.get(Dia)
    cantidad = len(df[df['release_date'].dt.dayofweek == dia_nro])

    return {'dia':Dia, 'cantidad':cantidad}

In [37]:
cantidad_filmaciones_dia('lunes')

{'dia': 'lunes', 'cantidad': 3501}

In [38]:
def score_titulo(titulo_de_la_filmacion: str):
    # Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
    #Ejemplo de retorno: La película X fue estrenada en el año X con una puntuación/popularidad de X
    
    titulo = titulo_de_la_filmacion.lower()
    result = df[df['title'].str.lower() == titulo]
    if len(result) == 0:
        return 'La pelicula no existe'
    
    anio = result['release_year']
    anio = anio.to_list()
    anio = anio[0]
    
    popularidad = result['popularity']
    popularidad = popularidad.to_list()
    popularidad = popularidad[0]
    
    return {'titulo':titulo, 'anio':anio, 'popularidad':popularidad}

In [39]:
score_titulo('Run Silent, Run Deep')

{'titulo': 'run silent, run deep', 'anio': 1958, 'popularidad': 2.73837}

In [40]:
def votos_titulo(titulo_de_la_filmacion: str):
    # Se ingresa el título de una filmación esperando como respuesta el título, 
    # la cantidad de votos y el valor promedio de las votaciones. La misma variable 
    # deberá contar con al menos 2000 valoraciones, caso contrario, debemos contar
    # con un mensaje avisando que no cumple esta condición y que por ende, 
    # no se devuelve ningun valor.
    # Ejemplo de retorno: La película X fue estrenada en el año X. 
    # La misma cuenta con un total de X valoraciones, con un promedio de X

    titulo = titulo_de_la_filmacion.lower()
    result = df[df['title'].str.lower() == titulo]
    if len(result) == 0:
        return 'La pelicula no existe'
    
    vote_count = result['vote_count']
    vote_count = vote_count.to_list()
    vote_count = vote_count[0]
    
    vote_average = result['vote_average']
    vote_average = vote_average.to_list()
    vote_average = vote_average[0]
    
    anio = result['release_year']
    anio = anio.to_list()
    anio = anio[0]
    
    if vote_count < 2000:
        return 'La misma filmacion deberá contar con al menos 2000 valoraciones'
    
    return {'titulo':titulo, 'anio':anio, 'voto_total':vote_count, 'voto_promedio':vote_average}

In [41]:
votos_titulo('This Is the End')

{'titulo': 'this is the end',
 'anio': 2013,
 'voto_total': 2394.0,
 'voto_promedio': 6.2}

In [42]:
def get_actor(nombre_actor:str) :
    # Se ingresa el nombre de un actor que se encuentra dentro de un dataset 
    # debiendo devolver el éxito del mismo medido a través del retorno. 
    # Además, la cantidad de películas que en las que ha realizado y el 
    # promedio de retorno. La definición no deberá considerar directores.
    # Ejemplo de retorno: El actor X ha recibido de X cantidad de filmaciones, 
    # el mismo ha obtenido un retorno de X con un promedio de X por filmación

    actor = nombre_actor.lower()
    # actor = 'Tom Hanks'.lower()
    result = df[df['actor'].str.lower().str.contains(actor, regex=True, na=False)]
    if len(result) == 0:
        return 'El actor no existe'
    
    cantidad_filmaciones = len(result)
    retorno_total = result['revenue'].sum() / result['budget'].sum()
    retorno_promedio = result['return'].mean()

    return {'actor':nombre_actor, 'cantidad_filmaciones':cantidad_filmaciones, 'retorno_total':retorno_total, 'retorno_promedio':retorno_promedio}   

In [43]:
get_actor('Tom Hanks')

{'actor': 'Tom Hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 3.9609895036874625,
 'retorno_promedio': 2.5190069866929914}

In [44]:
def get_director(nombre_director: str): 
    # Se ingresa el nombre de un director que se encuentra dentro de un 
    # dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    # Además, deberá devolver el nombre de cada película con la fecha de 
    # lanzamiento, retorno individual, costo y ganancia de la misma.

    director = nombre_director.lower()
    result = df[df['director'].str.lower().str.contains(director, regex=True, na=False)]
    if len(result) == 0:
        return 'El director no existe'

    retorno_total_director = result['revenue'].sum() / result['budget'].sum()
    peliculas = result[['title', 'release_year', 'return', 'budget', 'revenue']].to_dict('records')
    
    return {'director':nombre_director, 'retorno_total_director':retorno_total_director, 'peliculas': peliculas}

In [45]:
get_director('John Lasseter')

{'director': 'John Lasseter',
 'retorno_total_director': 4.028598760714286,
 'peliculas': [{'title': 'Toy Story',
   'release_year': 1995,
   'return': 12.4518011,
   'budget': 30000000.0,
   'revenue': 373554033.0},
  {'title': "A Bug's Life",
   'release_year': 1998,
   'return': 3.027157158333333,
   'budget': 120000000.0,
   'revenue': 363258859.0},
  {'title': 'Toy Story 2',
   'release_year': 1999,
   'return': 5.526298544444445,
   'budget': 90000000.0,
   'revenue': 497366869.0},
  {'title': 'Luxo Jr.',
   'release_year': 1986,
   'return': 0.0,
   'budget': 0.0,
   'revenue': 0.0},
  {'title': 'Cars',
   'release_year': 2006,
   'return': 3.849859575,
   'budget': 120000000.0,
   'revenue': 461983149.0},
  {'title': 'Cars 2',
   'release_year': 2011,
   'return': 2.79926198,
   'budget': 200000000.0,
   'revenue': 559852396.0},
  {'title': 'Tin Toy',
   'release_year': 1988,
   'return': 0.0,
   'budget': 0.0,
   'revenue': 0.0},
  {'title': "Red's Dream",
   'release_year': 1

In [46]:
def recomendacion_previo():
    global cosine_sim, df_ml, indices
    # De las caracteristicas de genero, actor, director y overview
    nro_registros = 5000
    features = ['_genres', 'director', 'actor', 'overview', 'title']
    df_ml = df[features].head(nro_registros)
    df_ml = df_ml.dropna(how='any', subset=features)
    import nltk
    import re

    # stopwords
    #nltk.download('stopwords')

    stemmer = nltk.SnowballStemmer("english")

    from nltk.corpus import stopwords
    import string

    stopword = set(stopwords.words("english"))

    # Definimos función de limpieza
    def clean_data(text):
        text = str(text).lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = [word for word in text.split(' ') if word not in stopword]
        text=" ".join(text)
        text = [stemmer.stem(word) for word in text.split(' ')]
        text=" ".join(text)
        return text    

    for feature in features:
        if feature not in  ['title']:
            df_ml[feature] = df_ml[feature].apply(clean_data)

    # Ahora podemos crear nuestra 'sopa de metadatos', que es una cadena que contiene
    # todos los metadatos que queremos alimentar a nuestro vectorizador
    def create_soup(x):
        return x['_genres'] + ' ' + x['director'] + ' ' + x['actor'] + ' ' + x['overview']        
    
    df_ml['soup'] = df_ml.apply(create_soup, axis=1)
    
    # Import CountVectorizer and create the cuount matrix
    from sklearn. feature_extraction.text import CountVectorizer

    count = CountVectorizer()
    count_matrix = count.fit_transform(df_ml['soup'])

    # Compute the cosine similarity matrix based on the count_matrix
    from sklearn.metrics.pairwise import cosine_similarity

    cosine_sim = cosine_similarity(count_matrix, count_matrix)

    # Restablece el indice de nuestro Dataframe principal y construyo el mapeo inverso como antes
    df_ml = df_ml.reset_index()
    indices = pd.Series(df_ml.index, index=df_ml['title'])                      
    

In [47]:
recomendacion_previo()

In [48]:
def recomendacion(titulo: str):
    # Ingresas un nombre de pelicula y te recomienda las similares en una lista
    try:
        idx = indices[titulo]
    except:
        return 'La pelicula no existe'
  
    # Obtengo las puntuaciones de similitud por pares de todas las peliculas con esa pelicula
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordene las peliculas segun las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obten las puntuaciones de las 5 peliculas mas similares
    sim_scores = sim_scores[1:6]

    # Obtenga los indices de las peliculas
    movie_indices = [i[0] for i in sim_scores]

    # Devuelve el top 5 de peliculas similares
    peliculas = df_ml['title'].iloc[movie_indices].to_list()
    
    return {'lista recomendada': peliculas}

In [49]:
recomendacion('Born to Win')

{'lista recomendada': ['Kissing a Fool',
  'Go',
  'Intimacy',
  'Suicide Kings',
  'The Killing']}